<a href="https://colab.research.google.com/github/vhrique/anne_ptbr/blob/development/04_Arquiteturas_Especializadas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Autoencoders

## Variational Autoencoders

# Visão Computacional

## Detecção de Objetos

### R-CNN (2014)

### Fast R-CNN (2015)

### Faster R-CNN (2015)

### YOLO (2015 - ...)

### RetinaNet (2017)

### DETR (2020)

## Segmentação de Imagens

### FCN (2015)

### U-Net (2015)

### DeepLab (2015 - ...)

### Mask R-CNN (2017)

### SAM (2023)

## Geração e Transformação de Imagens

### Generative Adversarial Networks

### StyleGAN


### CycleGAN

### PixelRNN

# Processamento de Linguagem Natural

## Seq2Seq